In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,14957451.67,32.79,41.35,-34.80,-0.30,-1.41,-0.07
4,32.79,1010887.07,-16.12,-8.94,4.97,0.59,0.10
5,41.35,-16.12,418478.55,1.36,2.34,1.47,-0.12
6,-34.80,-8.94,1.36,106004.06,-0.14,-0.04,-0.04
7,-0.30,4.97,2.34,-0.14,20604.05,0.11,0.03
8,-1.41,0.59,1.47,-0.04,0.11,1954.03,0.01
9,-0.07,0.10,-0.12,-0.04,0.03,0.01,94.97


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00001,0.00002,-0.00003,-0.00000,-0.00001,-0.00000
4,0.00001,1.00000,-0.00002,-0.00003,0.00003,0.00001,0.00001
5,0.00002,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,-0.00003,-0.00003,0.00001,1.00000,-0.00000,-0.00000,-0.00001
7,-0.00000,0.00003,0.00003,-0.00000,1.00000,0.00002,0.00002
8,-0.00001,0.00001,0.00005,-0.00000,0.00002,1.00000,0.00002
9,-0.00000,0.00001,-0.00002,-0.00001,0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

93.51313213782753

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.19655501e+13 1.43136655e+11 2.32687549e+10 1.40397249e+09
 4.70595311e+07 3.18192348e+05 2.70036038e-27]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999924,-0.011309,-0.004659,-0.001182,-0.000229,-0.000022,-0.000001
1,0.010770,0.994891,-0.098269,-0.020083,-0.003848,-0.000365,-0.000018
2,0.005633,0.096548,0.992800,-0.069561,-0.012464,-0.001176,-0.000058
3,0.001768,0.026351,0.066289,0.995110,-0.068018,-0.006129,-0.000298
4,0.000460,0.006799,0.016451,0.066536,0.996456,-0.048228,-0.002311
5,0.000065,0.000963,0.002325,0.009197,0.047513,0.997637,-0.048712
6,0.000006,0.000094,0.000227,0.000895,0.004602,0.048541,0.998810


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[7.552835982571615e-05,
 0.005108570470280238,
 0.007200339834251168,
 0.004889901027142152,
 0.003543977878781135,
 0.0023634190281126566,
 0.0011898391393819763]